In [4]:
kernel_code = """
__global int hws[50];

  
float mean(__global int* x, int n) {
    float sum = 0.0f;
    for (int i = 0; i < n; i++) {
        sum += x[i];
    }
    return sum / n;
}

float std_dev(__global float* x, float x_bar, int n) {
    float sum = 0.0f;
    for (int i = 0; i < n; i++) {
        sum += (x[i] - x_bar) * (x[i] - x_bar);
    }
    return sqrt(sum);
}

float cov(__global float* x, float x_bar, __global float* y, float y_bar, int n) {
    float sum = 0.0f;
    for (int i = 0; i < n; i++) {
        sum += (x[i] - x_bar) * (y[i] - y_bar);
    }
    return sum / n;
}
__kernel void compute_cpa(
    __global const float* trace_array,
    __global const uchar *textin_array,
    __global float *cpaoutput_array,
    int n_traces)
{
   // Define S-box here
     __constant uchar sbox[256] = { 
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76, 
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0, 
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15, 
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75, 
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84, 
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF, 
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8, 
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2, 
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, 
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08, 
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e, 
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16  
    };

    // Get thread IDs
    const int bnum = get_global_id(0);
    const int kguess = get_global_id(1);

    // Declare variables
    float correlation, cpaoutput;
    float maxcpa[256] = {0};
    float cparefs[16] = {0};
    int bestguess[16] = {0};
    float hws_bar;
    float t_bar = 0.0f, o_t = 0.0f, o_hws = 0.0f;

    // Compute t_bar
    for (int i = 0; i < n_traces; i++) {
        t_bar += trace_array[i * 5000 + bnum];
    }
    t_bar /= n_traces;

    // Compute o_t
    for (int i = 0; i < n_traces; i++) {
        o_t += pow(trace_array[i * 5000 + bnum] - t_bar, 2);
    }
    o_t = sqrt(o_t);

    
    for (int bnum_count = 0; bnum_count < 16; bnum_count++) {
        for (int kguess_count = 0; kguess_count < 256; kguess_count++) {
            for (int i = 0; i < n_traces; i++) {              
                int hw = __builtin_popcount(sbox[textin_array[i * 16 + bnum_count] ^ kguess_count]);
                hws[i] =  hw;
                
            }
            
            hws_bar = mean(hws, n_traces);
            
            
            o_hws = std_dev(hws, hws_bar, n_traces);
            cpaoutput_array[ bnum_count] = o_hws;
            correlation = cov(trace_array, t_bar, hws, hws_bar, n_traces);
            cpaoutput = correlation / (o_t * o_hws);
            
            int index = bnum_count * 256 + kguess_count;
            if (fabs(cpaoutput) > cparefs[bnum_count]) {
                cparefs[bnum_count] = fabs(cpaoutput);
                bestguess[bnum_count] = kguess_count;
            }
            
        }

    }
}
"""

In [5]:
import numpy as np
import pyopencl as cl

# Initialize OpenCL context, command queue, and device
platforms = cl.get_platforms()
devices = [device for platform in platforms for device in platform.get_devices()]
ctx = cl.Context(devices)
queue = cl.CommandQueue(ctx, devices[0])

# Create input and output numpy arrays

aes_traces_50_tracedata = np.load(r"traces/lab4_2_traces.npy")
aes_traces_50_textindata = np.load(r"traces/lab4_2_textin.npy")


trace_array = aes_traces_50_tracedata 
textin_array = aes_traces_50_textindata
cpaoutput_array = np.empty([16, ], dtype=np.float32)

# Set kernel arguments
n_traces = 50
global_size = (5000, )

# Create input and output OpenCL buffers
mf = cl.mem_flags
trace_array_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=trace_array)
textin_array_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=textin_array)
cpaoutput_array_buf = cl.Buffer(ctx, mf.WRITE_ONLY, cpaoutput_array.nbytes)

# Compile kernel and execute on the device
prg = cl.Program(ctx, kernel_code).build()
prg.compute_cpa(queue, global_size, None, trace_array_buf, textin_array_buf, cpaoutput_array_buf, np.int32(n_traces))
# Copy results from OpenCL buffer to numpy array

cl.enqueue_copy(queue, cpaoutput_array, cpaoutput_array_buf)

In [6]:
#np.set_printoptions(threshold=np.inf, suppress=True)
print("output:", cpaoutput_array)

output: [28.56711  30.688423 27.435741 27.435741 29.274214 27.86001  30.405584
 28.001423 30.829851 27.152905 29.55707  27.29432  28.284271 29.83991
 30.405584 27.86001 ]


In [ ]:
from tqdm import trange

# Compare results
t_bar = np.sum(trace_array, axis=0) / len(trace_array)
o_t = np.sqrt(np.sum((trace_array - t_bar)**2, axis=0))

cparefs = [0] * 16 #put your key byte guess correlations here
bestguess = [0] * 16 #put your key byte guesses here

def mean(X):
    return np.sum(X, axis=0)/len(X)

def std_dev(X, X_bar):
    return np.sqrt(np.sum((X-X_bar)**2, axis=0))

def cov(X, X_bar, Y, Y_bar):
    return np.sum((X-X_bar)*(Y-Y_bar), axis=0)

sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def aes_internal(inputdata, key):
    return sbox[inputdata ^ key]

HW = [bin(n).count("1") for n in range(0, 256)]
       
#print("hws:", t_bar)
#print("hws" ,cpaoutput_array .shape, len(cpaoutput_array ))
print("output:", cpaoutput_array)
print("Results are the same:", np.allclose(t_bar, cpaoutput_array))